In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [4]:
# read data and skip 'date' colume
df = pd.read_csv('./dataset/netflix-prize-data/combined_data_1.txt', 
                 header = None, names = ['user', 'rating','date'], 
                 usecols = [0,1,2])

In [6]:
def slice_movie(df):
    # find empty rows for slice movies 
    # dataset using movie id as user name for splite rating of each movies
    movies_interval = df[df['rating'].isna()]['user'].reset_index()
    movies_interval['index'] = movies_interval['index'].astype(int)
    
    # create list accroding to movie interval by np
    movie_id = 1
    movie_np = []
    last = None
    for i,r in movies_interval.iterrows():
        if last != None: 
            temp = np.full((1,r['index']-last-1), movie_id)
            movie_np = np.append(movie_np, temp)
            movie_id +=1
        last = r['index']
    # add last movie
    temp = np.full((1,len(df)-last-1), movie_id)
    movie_np = np.append(movie_np, temp)
    
    # removie movie slice row, and combine movie colume
    df = df[pd.notnull(df['rating'])]
    df['movie'] = movie_np.astype(int)
    
    return df

In [7]:
def get_users_df(df):
    # get user list and their rating count
    return df.groupby('user')['user'].agg(['count']).reset_index()

In [40]:
def filter_data_by_user(df,users_df, frac=0.1,count=50):
    # get test users
    test_users_df = users_df[users_df['count']>count].sample(frac=frac)
    # filter user by rating count
    cdf = df.merge(users_df, left_on='user',right_on='user')
    cdf = cdf[cdf['count']>count][['user','movie','rating']]
    
    train_set = cdf[~(cdf['user'].isin(test_users_df['user']))]
    test_set = cdf[cdf['user'].isin(test_users_df['user'])]
    return train_set, test_set

In [64]:
def filter_data_by_date(df,users_df, frac=0.1,count=100,year=2005,month=12,day=15):
    # get test users
    test_users_df = users_df[users_df['count']>count].sample(frac=frac)
    # filter user by rating count
    cdf = df.merge(users_df, on='user')
    cdf = cdf[cdf['count']>count][['user','movie','rating','date']]
    
#     train_set = cdf[~(cdf['user'].isin(test_users_df['user'])& (cdf['date']< pd.datetime(year,month,day)))]
    test_set = cdf[cdf['user'].isin(test_users_df['user'])& (cdf['date']> pd.datetime(year,month,day))]
    target_set = cdf[cdf['user'].isin(test_users_df['user'])]
    train_set = cdf[~(cdf.isin(test_set))]
    
    return train_set, test_set, target_set

In [9]:
sliced_df = slice_movie(df)

/Users/wujingyi/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [53]:
# sliced_df['date']= pd.to_datetime(sliced_df['date'],format="%Y/%mm/%dd")
# print(sliced_df['date'][:5])
# print(sliced_df[sliced_df['date']> pd.datetime(2005,12,1)])

In [65]:
sliced_df['rating']=sliced_df['rating'].astype(float)
sliced_df['movie']=sliced_df['movie'].astype(int)
sliced_df['user']=sliced_df['user'].astype(int)
print(sliced_df)

/Users/wujingyi/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/wujingyi/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


             user  rating       date  movie
1         1488844     3.0 2005-09-06      1
2          822109     5.0 2005-05-13      1
3          885013     4.0 2005-10-19      1
4           30878     4.0 2005-12-26      1
5          823519     3.0 2004-05-03      1
6          893988     3.0 2005-11-17      1
7          124105     4.0 2004-08-05      1
8         1248029     3.0 2004-04-22      1
9         1842128     4.0 2004-05-09      1
10        2238063     3.0 2005-05-11      1
11        1503895     4.0 2005-05-19      1
12        2207774     5.0 2005-06-06      1
13        2590061     3.0 2004-08-12      1
14           2442     3.0 2004-04-14      1
15         543865     4.0 2004-05-28      1
16        1209119     4.0 2004-03-23      1
17         804919     4.0 2004-06-10      1
18        1086807     3.0 2004-12-28      1
19        1711859     4.0 2005-05-08      1
20         372233     5.0 2005-11-23      1
21        1080361     3.0 2005-03-28      1
22        1245640     3.0 2005-1

/Users/wujingyi/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [66]:
# users_df = get_users_df(sliced_df)
# train_set, test_set = filter_data_by_user(sliced_df,users_df,frac=0.1,count=50)
train_set, test_set, target_set = filter_data_by_date(sliced_df,users_df)
print(test_set)

             user  movie  rating       date
3024      1080361    647     4.0 2005-12-16
3084      1080361   1338     4.0 2005-12-16
20463     1792741    301     4.0 2005-12-16
21638     1792741   3560     5.0 2005-12-22
32582     1493615   3612     5.0 2005-12-18
40628     1524964    445     3.0 2005-12-27
55638     1604238   1200     3.0 2005-12-23
67499     1394012      1     5.0 2005-12-19
67931     1394012   4057     4.0 2005-12-26
81374     2269844    631     4.0 2005-12-18
81435     2269844   2699     4.0 2005-12-18
90508       16272    758     3.0 2005-12-16
93372     2544219   4181     5.0 2005-12-18
93382     2544219   4474     3.0 2005-12-18
111752    1494196   3092     2.0 2005-12-25
117095    1547173    419     5.0 2005-12-27
117303    1547173   3528     3.0 2005-12-19
121386    2118461      2     4.0 2005-12-20
121388    2118461      4     3.0 2005-12-20
121390    2118461      6     4.0 2005-12-19
121391    2118461      7     3.0 2005-12-22
121392    2118461      8     3.0

In [61]:
# train_set

In [67]:
train_set.to_csv(r'./dataset/cleaned_train_5.csv',index=False,sep=',')
test_set.to_csv(r'./dataset/cleaned_test_5.csv',index=False,sep=',')
target_set.to_csv(r'./dataset/cleaned_target_5.csv',index=False,sep=',')

In [ ]:
def make_sprase_matrix(df,users_df):
    np_matrix = np.zeros((df.shape[0], df.shape[1]))
    users = list(users_df['user']) #用户列表

    for user in users:
        user_list = df[df['user']== user] #获取全部用户评分
        np_matrix[i][user_list['movie'] -1] = user_list['rating'] #按index插入评分

    return sprase_matrix

In [ ]:
sprase_train = make_sprase_matrix(train_set,users_df)
np.savetxt('dataset/sprase_train_2.txt',sprase_train)
sprase_test = make_sprase_matrix(test_set,users_df)
np.savetxt('dataset/sprase_test_2.txt',sprase_test)